In [7]:
import pandas as pd
import os
from concat_quoter_utc_toFeather import ConcatQuoterUtcToFeather

In [8]:
# Folder for binary df
folder_name = 'temp_df'
file_name = 'df.feather'
file_path = os.path.join(folder_name, file_name)

if not os.path.exists(file_path):
    create_feather_file = ConcatQuoterUtcToFeather(folder_name, file_name)
    create_feather_file.concat_quoter_utc()

In [9]:
df = pd.read_feather(file_path)

In [10]:
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85705211 entries, 0 to 85705210
Data columns (total 2 columns):
 #   Column  Dtype                           
---  ------  -----                           
 0   Time    datetime64[ns, Europe/Stockholm]
 1   Value   float64                         
dtypes: datetime64[ns, Europe/Stockholm](1), float64(1)
memory usage: 1.3 GB
None


In [11]:
display(df.head(20))

,Time,Value
0,2020-12-31 23:00:00+01:00,49.999
1,2020-12-31 23:00:01+01:00,49.999
2,2020-12-31 23:00:02+01:00,49.997
3,2020-12-31 23:00:03+01:00,49.995
4,2020-12-31 23:00:04+01:00,49.994
5,2020-12-31 23:00:05+01:00,49.991
6,2020-12-31 23:00:06+01:00,49.989
7,2020-12-31 23:00:07+01:00,49.987
8,2020-12-31 23:00:08+01:00,49.985
9,2020-12-31 23:00:09+01:00,49.982


In [6]:
display(df.loc[(~df['Value'].between(49.5,50.5)) & (df['Value'] != -1)])

,Time,Value
20952261,2021-09-02 12:04:19+02:00,49.47000
20952262,2021-09-02 12:04:20+02:00,49.48900
20952263,2021-09-02 12:04:21+02:00,49.44600
51135191,2022-08-21 20:13:05+02:00,50.50240
54074178,2022-09-24 20:36:12+02:00,49.42628
54074179,2022-09-24 20:36:13+02:00,49.45048
54074180,2022-09-24 20:36:14+02:00,49.45205
54074181,2022-09-24 20:36:15+02:00,49.47887
66503407,2023-02-17 16:09:59+01:00,49.46900
66503408,2023-02-17 16:10:00+01:00,49.43100


In [14]:
threshold = 49.5
under_threshold = (df['Value'] < threshold) & (df['Value'] != -1)
min_consecutive_seconds = 5
consecutive_occurrences = under_threshold.groupby((~under_threshold).cumsum()).cumcount() + 1

# Filter values based on the minimum consecutive seconds
filtered_values = df.loc[consecutive_occurrences >= min_consecutive_seconds]

# Add a buffer to include leading values
buffer_size = 3  # Adjust this to include more or fewer leading values
filtered_values_with_buffer = df.loc[df.index.isin(filtered_values.index.union(filtered_values.index - buffer_size))]

# Print the result
print(filtered_values_with_buffer)

                              Time     Value
54074178 2022-09-24 20:36:12+02:00  49.42628
54074181 2022-09-24 20:36:15+02:00  49.47887
66503407 2023-02-17 16:09:59+01:00  49.46900
66503408 2023-02-17 16:10:00+01:00  49.43100
66503409 2023-02-17 16:10:01+01:00  49.43200
66503410 2023-02-17 16:10:02+01:00  49.44700
66503411 2023-02-17 16:10:03+01:00  49.46300
66503412 2023-02-17 16:10:04+01:00  49.49100
72254416 2023-04-26 06:40:07+02:00  49.43386
72254417 2023-04-26 06:40:08+02:00  49.32441
72254418 2023-04-26 06:40:09+02:00  49.31911
72254419 2023-04-26 06:40:10+02:00  49.35835
72254420 2023-04-26 06:40:11+02:00  49.38340
72254421 2023-04-26 06:40:12+02:00  49.43063
72254422 2023-04-26 06:40:13+02:00  49.46975


In [8]:
df.head()

,Time,Value
0,2020-12-31 23:00:00+01:00,49.999
1,2020-12-31 23:00:01+01:00,49.999
2,2020-12-31 23:00:02+01:00,49.997
3,2020-12-31 23:00:03+01:00,49.995
4,2020-12-31 23:00:04+01:00,49.994


In [9]:
# Define the threshold value
threshold = 49.5

# Set 'Time' as the index
df.set_index('Time', inplace=True)

# Define the rolling window size
time_window = '10S'

# Calculate the rolling minimum within the specified window
df['rolling_min'] = df['Value'].rolling(time_window).min()

# Identify consecutive occurrences that meet the condition
df['Under_49.9_over_time'] = (df['rolling_min'] < threshold) & (df['Value'] != -1)

# Filter values based on the minimum consecutive seconds
min_consecutive_seconds = 5
filtered_values = df.loc[df['Under_49.9_over_time'].rolling(window=min_consecutive_seconds).sum() >= min_consecutive_seconds]

# Reset index for final output (optional)
filtered_values.reset_index(inplace=True, names="Time")

# Print the result
print(filtered_values[['Time', 'Value']])

                           Time   Value
0     2021-01-01 01:59:51+01:00  49.966
1     2021-01-01 01:59:52+01:00  49.964
2     2021-01-01 01:59:53+01:00  49.965
3     2021-01-01 01:59:54+01:00  49.964
4     2021-01-01 01:59:55+01:00  49.962
...                         ...     ...
19587 2023-08-31 18:14:33+02:00  50.004
19588 2023-08-31 18:14:34+02:00  49.998
19589 2023-08-31 18:14:35+02:00  49.995
19590 2023-08-31 18:14:36+02:00  49.990
19591 2023-08-31 18:14:37+02:00  49.981

[19592 rows x 2 columns]


In [ ]:
df_over = df[(df['Value'] > 50.1) & (df['Value'] != -1)]
count_over_50 = len(df_over)
print("The number of rows with 'Value' over 50.1 is:", count_over_50)

In [ ]:
df_under = df[(df['Value'] < 49.9) & (df['Value'] != -1)]
count_under_50 = len(df_under)
print("The number of rows with 'Value' under 49.9 is:", count_under_50)

#### Disturbance active over time

In [ ]:
df.set_index('Time', inplace=True)
time_window = '5S'
df['Over_50.1_over_time'] = (df['Value'].rolling(time_window).max() > 50.1) & (df['Value'] != -1)
df['Under_49.9_over_time'] = (df['Value'].rolling(time_window).min() < 49.9) & (df['Value'] != -1)
ValOverUnder = (df[['Value', 'Over_50.1_over_time', 'Under_49.9_over_time']])
ValOverUnder

In [ ]:
print(len(df[df['Over_50.1_over_time'] == True]))

In [ ]:
print([df['Under_49.9_over_time'] == True])